In [1]:
from time import time
import numpy as np
import pandas as pd
# Парсинг
import requests
from bs4 import BeautifulSoup as BS
# Обработка текста
import re
from validate_email_address import validate_email
from itertools import groupby
# Работа с файлами
import json
import os
#Вывод прогресса
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output

KeyboardInterrupt: 

In [ ]:
# url1 = 'Teplypollipetsk.ru'
# url2 = 'http://иномарка46.рф'
# url3 = 'www.tochkamira.ru'
# print(cheek_url(url1))
# print(cheek_url(url2))
# print(cheek_url(url3))

In [ ]:
#Работоспособность сайта
#Список найденных неправильных протоколов
bad_list = ['https','http','hhtp://','hhtps://','hhtps://','hpp:','hpps:','hppt','hppts','htt','htth','htto','htpps','^','httpp',
            'httt','ttps']
def cheek_url(url):
    s = url
    # Удаление протоколов из ссылки
    for i in bad_list:
        s = s.replace(i,'')
    # Удаление первого символа из спец символов в списке
    while s[0] in ['/','.',';',':',',']:
        s = s[1:]
    # Удаление последнего символа из спец символов в списке
    while s[-1] in ['/','.',';',':',',']:
        s = s[:-1]
    url = s
    s_c = 'bad_url'
    
    try:
        url = f'https://{s}'
        # Попытка обратиться к сайту с обновленным защищенным протоклом  
        response = requests.get(url,timeout = 60)
        s_c = response.status_code
        if s_c == 200:
            return url, s_c
        elif s_c != 'bad_url':
            return url, s_c
    except:
        try:
            # Попытка обратиться к сайту с обновленным незащищенным протоклом  
            url = f'http://{s}'
            response = requests.get(url,timeout = 60)
            s_c = response.status_code
            if s_c == 200:
                return url, s_c
            elif s_c != 'bad_url':
                return url, s_c
        except:
            return url, s_c

In [ ]:
#Найти все ссылки на странице 
def get_all_links(page):
    try:
        HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'}
        html_text = requests.get(page, headers=HEADERS, timeout = 60).text
        soup = BS(html_text)
        arr = []
        for link in soup.find_all('a', href=True):
            l = link['href']

            if l.find("//") == -1 :
                try:
                    if l[0] != "/":
                       l = page + '/' + l 
                    else:
                        l = page + l
                except:
                    pass
            if l!='':
                arr.append(l)
        return [el for el, _ in groupby(arr)]
    except:
        return[]

In [ ]:
# Поиск триггеров в тексте
#new_triger_list_pay = ['битное шифрование','платежный шлюз','сбербанк']
#triger_list_ord = ["возврат","брак","обмен","ненадлежащего качества"]

new_triger_list_pay_1 = r'битное шифрование'
new_triger_list_pay_2 = r'платежн.{2,3} шлюз.{1,2}|сбербанк.{1,2}'

new_triger_pers = r'''обработ.{1,5}персональн.{1,5}данных|политик.{1,5} конфиденциальности|
политика и защита персональных данных|политика обработки персональных данных|политика по обработке персональных данных|
предоставлена третьим лицам|конфиденциальность|152-фз'''

triger_list_ord =r"возврат|брак|обмен|ненадлежащего качества"

triger_list_transfer = r'переводом на карту|перевод по карте|перевести на карту'

def new_find_triggers(text):    
    new_trig_list_pay = ''
    new_trig_list_pers = ''
    trig_list_ord = ''
    
    new_score_pay = 0
    score_ord = 0
    new_score_pers = 0
    score_transfer = 0
    #Оплата
    pay_find1 = re.findall(new_triger_list_pay_1,text)
    pay_find2 = re.findall(new_triger_list_pay_2,text)
    if len(pay_find1) > 0:
        new_score_pay = 1
        new_trig_list_pay = ' , '.join(str(n) for n in pay_find1 )
    elif len(pay_find2) == 2:
        new_score_pay = 1
        new_trig_list_pay = ' , '.join(str(n) for n in pay_find2 )
    #персональные данные    
    pers_find = re.findall(new_triger_pers,text)    
    if len(pers_find) > 0:
        new_score_pers = 1
        new_trig_list_pers = ' , '.join(str(n) for n in pers_find )
    #Заказ
    ord_find = re.findall(triger_list_ord,text)
    if len(ord_find) > 0:
        score_ord = 1
        trig_list_ord = ' , '.join(str(n) for n in ord_find )
    #Перевод на карту            
    trans_find = re.findall(triger_list_transfer,text)    
    if len(trans_find) > 0:
        score_transfer = 1
                
    return new_trig_list_pay,new_trig_list_pers, new_score_pay,new_score_pers,trig_list_ord, score_ord, score_transfer

In [ ]:
#Получить все эмейлы их текста
def find_all_emails(text):
    alphabet = ('АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя')
    emails = re.findall(r'[\w.-]+@[\w\.-]+\.[\w\.]+',text)
    emails = list(set(emails))
    for i in range(len(emails)):
        #Удалить русские буквы 
        for char in emails[i]:
            if char in alphabet:
                emails[i] = emails[i].replace(char,'')
        #удалить первые и последние точки      
        while emails[i][0] == '.':
            emails[i] = emails[i][1:]
        while emails[i][-1] == '.':
            emails[i] = emails[i][:-1]    
    return emails

In [ ]:
# Првоерить эмейлы 
def cheсk_email(emailinput):
    emailcheck = validate_email(emailinput)  
    #print(emailcheck)
    if emailcheck:
        return True
    else:
        return False

In [ ]:
#Проверка эмейлов и классификация на хорошие и плохие
def find_check_emails(text):
    emails_start_time = time()
    emails_g = ''
    emails_b = ''
    n_g = 0
    n_b = 0
    emails_list = []
    
    for email in find_all_emails(text):
        if emails_start_time - time() >= 300:
            return 'time','time','time','time'
        if not email in list(emails_list):
            emails_list.append(email)
            cheсk = cheсk_email(email)
            
            if cheсk:
                emails_g += f'{email} , '
                n_g += 1
            else:
                emails_b += f'{email} , '
                n_b += 1
                
    return emails_g,emails_b,n_g,n_b

In [ ]:
# Поиск ИНН
INN_re = re.compile(r'\d{10}|\d{12}')
#ИНН\x20*
df_reestr = pd.read_csv('recheck_urls.csv',index_col = 0)
def find_inn(text,URL):
    inn_start_time = time()
    final = ''
    n_inn = 0
    inn_list = list(set(INN_re.findall(text)))
    same = df_reestr[df_reestr.URL == URL]
    INN_list = list(same.INN)
    
    for i in  INN_list:
        if inn_start_time - time() >= 300:
            return 'time','time'
        i = str(i)
        if i in inn_list:
            final += f'{i} , '
            n_inn +=1

    return final, n_inn

In [ ]:
#Поиск номеров телефонов
phone_re = re.compile(r'(\d{1}[-\.\s]??\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
# phone_re = re.compile(r"(?:(?:8|\+7)[\- ]?)?(?:\(?\d{3}\)?[\- ]?)?[\d\- ]{7,10}")
# #phone_re = re.compile(r"((8|\+7)[\- ]?)?(\(?\d{3}\)?[\- ]?)?[\d\- ]{7,10}")
# #phone_re = re.compile(r'(?:(?:\+?1\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?')
#phone_re = re.compile(r'\b\+?[7,8](\s*\d{3}\s*\d{3}\s*\d{2}\s*\d{2})\b')

def find_phone(text):
    ph_start_time = time()
    num_phone = 0
    
    text_ph = text.lower()
    text_ph = text_ph.replace(' ','')
    text_ph = text_ph.replace('(','')
    text_ph = text_ph.replace(')','')
    text_ph = text_ph.replace('-','')
    text_ph = text_ph.replace('+','')
    results = ( phone_re.findall(text_ph) )
    new_res = ''
    results= list(set(results))
    for ph in results:
        if ph_start_time - time() >= 300:
            return 'time','time'
        ph = str(ph).replace(' ','')
        if len(ph)>=10 and ph[0] in ['7','8']:
        #if len(ph) >=5:
            new_res += f'{ph} , '
            num_phone +=1

    return  new_res, num_phone

In [ ]:
#Поиск в тексте триггеров авторизации и 404 ошибки
login_r = 'логин|пароль|личный кабинет|войти|регистрация|имя пользователя'
NotFound_r = '404|not found'
  
def log_404(text):
    log = re.findall(login_r,text)    
    if len(log) > 0:
        log_s = 1
    else:
        log_s = 0
            
    N_F = re.findall(NotFound_r,text)    
    if len(N_F) > 0:
        N_F_s = 1
    else:
        N_F_s = 0
                        
    return log_s, N_F_s

In [ ]:
#Кол-во символов в тексте без пробелов 
def len_page(text):
    text = text.replace(' ','')
    return len(text)

In [ ]:
df_all = pd.read_csv('ALL_02_new_scrchk.csv',index_col = 0,low_memory=False)
df_all.columns

In [ ]:
col_to_clear = ['URL_x','name', 'URL_y', 'transfer', 'respons', 'pay', 'pers', 'order',
       'contacts', '10KB', 'total', 'LEN', 'LOG', 'NF_404', 'num_char','Status']
for col in col_to_clear:  
    df_all[col] = np.NaN
df_all

In [ ]:
%%time
i = 0
start_time = time()
change = 0

df_work = df_recheck.copy()

len_df = len(list(df_work.itertuples(index=True, name='Pandas')))

ProgBar_all = IntProgress(min=0, max=len_df, value=i)


HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'}

for row in df_work.itertuples(index=True, name='Pandas'):
    #text = ' '
    print(f'i = {i}/{l1}, URL = {row.URL_old}')
    ProgBar_all.value = i
    display(ProgBar_all)
    
    new_url, status = cheek_url(row.URL_old)
    df_work.loc[row.Index, 'URL_x'] = new_url
    df_work.loc[row.Index, 'Status'] = status
    if str(status) != '200':
        i+=1
        continue
    df_work.loc[row.Index, 'URL_y'] = new_url
    j = 0
    list_url = get_all_links(new_url)
    list_url.insert(0,new_url)
    len_url = len(list_url)
    ProgBar_page = IntProgress(min=j, max=len_url, value=j)
    
    for url in list_url:
        ProgBar_page.value = j
        if url.endswith('.jpg') or url.endswith('.pdf') or url.endswith('.exe') or url.endswith('.mp4') or url.endswith('.jpeg'):
            j+=1
            continue
        clear_output(True)
        print(f'i = {i}/{l1}, URL = {new_url}')
        print(f"page {j} of {len_url}. URL = {url} . Time = {(time() - start_time) // 60}")
        ProgBar_page.value = j
        display(ProgBar_page)
        j+=1
        try:
            html = requests.get(url,headers=HEADERS,timeout = 60).text
            soup = BS(html)
            text =  soup.body.get_text().lower()
        except:
            continue
        
        new_trig_list_pay,new_trig_list_pers, new_score_pay,new_score_pers,trig_list_ord, score_ord, score_transfer = new_find_triggers(text)
        log_s, N_F_s = log_404(text)
        n_char = len_page(text)
        
        df_work.loc[row.Index, 'transfer'] = df_work.loc[row.Index, 'transfer'] or score_transfer
        df_work.loc[row.Index, 'pay'] = df_work.loc[row.Index, 'pay'] and new_score_pay == 0
        df_work.loc[row.Index, 'pers'] = df_work.loc[row.Index, 'pers'] and new_score_pers == 0   
        df_work.loc[row.Index, 'order'] = df_work.loc[row.Index, 'order'] and score_ord == 0
        
        
    i+=1

In [ ]:
['URL_x','name', 'URL_y', 'transfer', 'respons', 'pay', 'pers', 'order',
       'contacts', '10KB', 'total', 'LEN', 'LOG', 'NF_404', 'num_char']